In [ ]:
%run -i rename.py

In [ ]:
from moire_screen_gan import Net
m = Net()

In [ ]:
modelPath = '../model/demoire/moire_screen_gan.pth'
weights = torch.load(modelPath, map_location='cpu')

getW = getNames(weights)
ccW = cc(weights)

In [ ]:
#modelPath = 'moire_t.pth'
torch.save(weights, modelPath, pickle_protocol=4)

In [ ]:
weights = torch.load('moire_t.pth', map_location='cpu')

In [ ]:
list(weights.keys())

In [ ]:
w1 = m.state_dict()
list(w1.keys())

In [ ]:
rsts = [
    ((r'branch.\.convt_F..\.',), 'ca1.', '0.ca.', None, None),
    ((r'branch.\.convt_F..\.',), 'ca2.', '1.ca.', None, None),
    ((r'branch.\.convt_F..\.',), 'relu1.', '0.relu.', None, None),
    ((r'branch.\.convt_F..\.',), 'relu2.', '1.relu.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv1.', '0.conv1.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv2.', '0.conv2.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv3.', '1.conv1.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv4.', '1.conv2.', None, None),
    ((r'branch.\.convt_F..\.[01]\.',), None, None, '', '0.'),
    ((r'branch.\.u.\.',), 'conv.', '0.', None, None),
    ((r'branch.\.u.\.',), 'prelu.', '2.', None, None),
    ((r'branch.\.non_local\.non_local\.[^.]+\.',), '0.', '', None, None)
] + [
    reT(fm1(i, r'(branch.\.)convt_F1{}\.', 'convt_F1{}.', 'convt_F.{}.')) for i in range(8)
] + [
    reT(fm1(i, r'(branch.\.)s_conv{}\.', 's_conv{}.', 's_conv.{}.')) for i in range(8)
] + [
    reT(fm1(i, r'(branch.\.)u{}\.', 'u{}.', 'u.{}.')) for i in range(8)
] + [
    reT(fm1(i, r'()branch{}\.', 'branch{}.', 'branches.{}.')) for i in range(5)
] + [
    reT(fm1(i, r'()scale{}\.', 'scale{}.', 'scales.{}.')) for i in range(5)
] + [
    reT(fm1(i, r'()down2_{}\.', 'down2_{}.', '_down2.{}.')) for i in range(4)
]

In [ ]:
removeRoot(weights, 'branch6')
removeRoot(weights, 'scale6')
removeRoot(weights, 'down2_5')
ccW(rsts)
m.load_state_dict(weights)

In [ ]:
torch.save(m.state_dict(), modelPath)